In [12]:
from sklearn.datasets import load_boston
import pandas as pd


In [13]:
boston = load_boston()
data = pd.DataFrame(boston.data)
data.columns = boston.feature_names

In [25]:
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [24]:
# Add label to dataset
data['PRICE'] = boston.target

In [29]:
data.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [31]:
import pickle
import numpy as np
import xgboost as xgb

In [80]:
train = pickle.load( open("Z:/LKS-CHART/Projects/physionet_sepsis_project/data/splits/split_0/train_preprocessed_measured.pkl", "rb"))
test = pickle.load( open("Z:/LKS-CHART/Projects/physionet_sepsis_project/data/splits/split_0/valid_preprocessed_measured.pkl", "rb"))


In [126]:
# Take small sample of dataset
train_subject = np.unique(train['id'])
train_subject_small = np.random.choice(train_subject, size = 50, replace = False)
train_small = train[train['id'].isin(train_subject_small)]

test_subject = np.unique(test['id'])
test_subject_small = np.random.choice(test_subject, size = 50, replace = False)
test_small = test[test['id'].isin(test_subject_small)]

# Create xgboost matrices
dtrain = xgb.DMatrix(train.drop(columns = ['SepsisLabel', 'filename', 'id']),
                    label = train['SepsisLabel'])
dtest = xgb.DMatrix(test.drop(columns = ['SepsisLabel', 'filename', 'id']),
                    label = test['SepsisLabel'])

In [136]:
# Set parameters

params = {
    'max_depth': 20,
    'eta': 0.01,
    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'min_child_weight': 10,
    'max_delta_step': 1,
    'silent': 0,
    'objective': 'binary:logistic',
    'eval_metric': "logloss"
}

watchlist = [(dtrain, 'train'), (dtest, 'eval')]
num_round = 100000

# Fit model
model = xgb.train(params, 
                  dtrain,
                  num_round,
                  watchlist,
                  early_stopping_rounds=10)

[04:48:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1740 extra nodes, 0 pruned nodes, max_depth=20
[0]	train-logloss:0.683766	eval-logloss:0.683921
Multiple eval metrics have been passed: 'eval-logloss' will be used for early stopping.

Will train until eval-logloss hasn't improved in 10 rounds.
[04:48:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1812 extra nodes, 0 pruned nodes, max_depth=20
[1]	train-logloss:0.674569	eval-logloss:0.674859
[04:48:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1848 extra nodes, 0 pruned nodes, max_depth=20
[2]	train-logloss:0.665549	eval-logloss:0.665984
[04:48:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1716 extra nodes, 0 pruned nodes, max_depth=20
[3]	train-logloss:0.656709	eval-logloss:0.657276
[04:48:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1732 extra nodes, 0 pruned nodes, max_depth=20
[4]	train-logloss:0.648039	eval-logloss:0.64875
[04:48:25] src/tree/updater_prun

[04:51:47] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2312 extra nodes, 0 pruned nodes, max_depth=20
[49]	train-logloss:0.379293	eval-logloss:0.385543
[04:51:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2350 extra nodes, 0 pruned nodes, max_depth=20
[50]	train-logloss:0.375194	eval-logloss:0.381584
[04:51:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2364 extra nodes, 0 pruned nodes, max_depth=20
[51]	train-logloss:0.371158	eval-logloss:0.377656
[04:52:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2470 extra nodes, 0 pruned nodes, max_depth=20
[52]	train-logloss:0.367199	eval-logloss:0.373798
[04:52:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2382 extra nodes, 0 pruned nodes, max_depth=20
[53]	train-logloss:0.363284	eval-logloss:0.369995
[04:52:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2584 extra nodes, 0 pruned nodes, max_depth=20
[54]	train-logloss:0.359436	eval-logloss:0.366245
[04:52:17]

[04:56:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2822 extra nodes, 0 pruned nodes, max_depth=20
[99]	train-logloss:0.229827	eval-logloss:0.241267
[04:56:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2720 extra nodes, 0 pruned nodes, max_depth=20
[100]	train-logloss:0.227706	eval-logloss:0.239236
[04:56:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2538 extra nodes, 0 pruned nodes, max_depth=20
[101]	train-logloss:0.225604	eval-logloss:0.237237
[04:56:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2814 extra nodes, 0 pruned nodes, max_depth=20
[102]	train-logloss:0.223547	eval-logloss:0.235262
[04:56:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2904 extra nodes, 0 pruned nodes, max_depth=20
[103]	train-logloss:0.221525	eval-logloss:0.233324
[04:56:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2600 extra nodes, 0 pruned nodes, max_depth=20
[104]	train-logloss:0.219503	eval-logloss:0.231407
[04:5

[04:59:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2896 extra nodes, 0 pruned nodes, max_depth=20
[149]	train-logloss:0.149133	eval-logloss:0.165504
[05:00:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2938 extra nodes, 0 pruned nodes, max_depth=20
[150]	train-logloss:0.147933	eval-logloss:0.164396
[05:00:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2984 extra nodes, 0 pruned nodes, max_depth=20
[151]	train-logloss:0.146756	eval-logloss:0.163316
[05:00:11] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2904 extra nodes, 0 pruned nodes, max_depth=20
[152]	train-logloss:0.14559	eval-logloss:0.162244
[05:00:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2982 extra nodes, 0 pruned nodes, max_depth=20
[153]	train-logloss:0.144439	eval-logloss:0.161186
[05:00:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2642 extra nodes, 0 pruned nodes, max_depth=20
[154]	train-logloss:0.143296	eval-logloss:0.160124
[05:0

[05:03:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3028 extra nodes, 0 pruned nodes, max_depth=20
[199]	train-logloss:0.102662	eval-logloss:0.123922
[05:04:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3086 extra nodes, 0 pruned nodes, max_depth=20
[200]	train-logloss:0.101965	eval-logloss:0.123314
[05:04:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2896 extra nodes, 0 pruned nodes, max_depth=20
[201]	train-logloss:0.101259	eval-logloss:0.122702
[05:04:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2676 extra nodes, 0 pruned nodes, max_depth=20
[202]	train-logloss:0.10056	eval-logloss:0.122106
[05:04:18] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2860 extra nodes, 0 pruned nodes, max_depth=20
[203]	train-logloss:0.099874	eval-logloss:0.121523
[05:04:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3188 extra nodes, 0 pruned nodes, max_depth=20
[204]	train-logloss:0.099193	eval-logloss:0.120956
[05:0

...

[05:11:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2586 extra nodes, 0 pruned nodes, max_depth=20
[299]	train-logloss:0.056983	eval-logloss:0.08771
[05:12:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2712 extra nodes, 0 pruned nodes, max_depth=20
[300]	train-logloss:0.056705	eval-logloss:0.087522
[05:12:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2582 extra nodes, 0 pruned nodes, max_depth=20
[301]	train-logloss:0.056426	eval-logloss:0.087335
[05:12:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2700 extra nodes, 0 pruned nodes, max_depth=20
[302]	train-logloss:0.05614	eval-logloss:0.087156
[05:12:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2538 extra nodes, 0 pruned nodes, max_depth=20
[303]	train-logloss:0.055862	eval-logloss:0.08697
[05:12:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2944 extra nodes, 0 pruned nodes, max_depth=20
[304]	train-logloss:0.055592	eval-logloss:0.086791
[05:12:

[05:15:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2452 extra nodes, 0 pruned nodes, max_depth=20
[349]	train-logloss:0.045239	eval-logloss:0.080593
[05:15:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2634 extra nodes, 0 pruned nodes, max_depth=20
[350]	train-logloss:0.045057	eval-logloss:0.080484
[05:16:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2408 extra nodes, 0 pruned nodes, max_depth=20
[351]	train-logloss:0.044856	eval-logloss:0.080378
[05:16:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2492 extra nodes, 0 pruned nodes, max_depth=20
[352]	train-logloss:0.044664	eval-logloss:0.080281
[05:16:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2450 extra nodes, 0 pruned nodes, max_depth=20
[353]	train-logloss:0.044471	eval-logloss:0.080184
[05:16:18] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2530 extra nodes, 0 pruned nodes, max_depth=20
[354]	train-logloss:0.044286	eval-logloss:0.080095
[05:

[05:19:48] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2356 extra nodes, 0 pruned nodes, max_depth=20
[399]	train-logloss:0.037144	eval-logloss:0.076851
[05:19:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2386 extra nodes, 0 pruned nodes, max_depth=20
[400]	train-logloss:0.03701	eval-logloss:0.076798
[05:19:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2370 extra nodes, 0 pruned nodes, max_depth=20
[401]	train-logloss:0.036875	eval-logloss:0.076746
[05:20:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2300 extra nodes, 0 pruned nodes, max_depth=20
[402]	train-logloss:0.036745	eval-logloss:0.076697
[05:20:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2160 extra nodes, 0 pruned nodes, max_depth=20
[403]	train-logloss:0.036608	eval-logloss:0.076641
[05:20:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2392 extra nodes, 0 pruned nodes, max_depth=20
[404]	train-logloss:0.036479	eval-logloss:0.076588
[05:2

[05:23:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2114 extra nodes, 0 pruned nodes, max_depth=20
[449]	train-logloss:0.031348	eval-logloss:0.075023
[05:23:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2078 extra nodes, 0 pruned nodes, max_depth=20
[450]	train-logloss:0.03124	eval-logloss:0.07501
[05:23:48] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2076 extra nodes, 0 pruned nodes, max_depth=20
[451]	train-logloss:0.031136	eval-logloss:0.074981
[05:23:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2198 extra nodes, 0 pruned nodes, max_depth=20
[452]	train-logloss:0.031045	eval-logloss:0.074955
[05:23:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2068 extra nodes, 0 pruned nodes, max_depth=20
[453]	train-logloss:0.030939	eval-logloss:0.074933
[05:24:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1994 extra nodes, 0 pruned nodes, max_depth=20
[454]	train-logloss:0.030835	eval-logloss:0.074905
[05:24

[05:27:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2074 extra nodes, 0 pruned nodes, max_depth=20
[499]	train-logloss:0.02702	eval-logloss:0.074341
[05:27:29] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2032 extra nodes, 0 pruned nodes, max_depth=20
[500]	train-logloss:0.026942	eval-logloss:0.074334
[05:27:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1820 extra nodes, 0 pruned nodes, max_depth=20
[501]	train-logloss:0.026865	eval-logloss:0.074323
[05:27:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1994 extra nodes, 0 pruned nodes, max_depth=20
[502]	train-logloss:0.026796	eval-logloss:0.074308
[05:27:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1984 extra nodes, 0 pruned nodes, max_depth=20
[503]	train-logloss:0.026718	eval-logloss:0.074301
[05:27:47] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1964 extra nodes, 0 pruned nodes, max_depth=20
[504]	train-logloss:0.026635	eval-logloss:0.074292
[05:2

In [143]:
search_results.shape[0]

32

In [137]:
model.save_model('xgb.model')

In [130]:
search_results = pd.read_csv("H:/LKS-CHART/projects/physionet-2019/david/xgboost/xgboost_search.csv")

In [134]:
search_results.sort_values(by = "eval_logloss", ascending = True)

,nrounds,max_depth,eta,subsample,colsample_bytree,min_child_weight,max_delta_step,silent,eval_logloss,v14,split
0,100000,20,0.010,0.75,0.75,10,1,1,0.101672,511,1
1,100000,10,0.010,0.75,0.75,10,10,1,0.101881,422,1
2,100000,7,0.010,0.75,0.75,10,10,1,0.101983,439,1
3,100000,10,0.010,0.75,0.75,10,1,1,0.102041,483,1
4,100000,10,0.001,0.75,0.75,10,1,1,0.102074,4784,1
5,100000,4,0.010,0.75,0.75,10,10,1,0.102165,446,1
6,100000,12,0.010,0.75,0.75,10,1,1,0.102182,502,1
7,100000,12,0.001,0.75,0.75,10,1,1,0.102204,4733,1
8,100000,7,0.001,0.75,0.75,10,1,1,0.102221,4667,1
9,100000,7,0.010,0.75,0.75,10,1,1,0.102312,477,1
